In [19]:
import numpy as np
from tqdm import tqdm

# Make sure this matches your worker observation builder
from hrl.worker_env import TableTennisWorker
from stable_baselines3.common.vec_env import DummyVecEnv
from config import Config

In [10]:
import numpy as np
from tqdm import tqdm
from stable_baselines3.common.vec_env import DummyVecEnv

from config import Config
from hrl.worker_env import TableTennisWorker

def reset_env(venv):
    result = venv.reset()
    return result[0] if isinstance(result, tuple) else result

def get_base_env(venv):
    """
    Unwrap DummyVecEnv / Monitor / VecNormalize
    down to the base gym env that has obs_dict.
    """
    env = venv.envs[0]
    while hasattr(env, "env"):
        env = env.env
    return env.unwrapped

def collect_relative_offsets(cfg, num_steps=30000):
    venv = DummyVecEnv([lambda: TableTennisWorker(cfg)])
    base_env = get_base_env(venv)

    dxs, dys, dzs = [], [], []
    dpxs, dpys = [], []
    dts = []

    reset_env(venv)
    for _ in tqdm(range(num_steps)):
        action = [venv.action_space.sample()]
        out = venv.step(action)

        obs_dict = base_env.obs_dict  # raw MyoSuite obs
        ball_pos = np.asarray(obs_dict["ball_pos"], np.float32)
        paddle_pos = np.asarray(obs_dict["paddle_pos"], np.float32)
        pelvis_xy = np.asarray(obs_dict["pelvis_pos"][:2], np.float32)
        t = float(obs_dict["time"])

        # Relative differences
        dx, dy, dz = paddle_pos - ball_pos
        dpx, dpy = pelvis_xy - ball_pos[:2]

        dxs.append(dx)
        dys.append(dy)
        dzs.append(dz)
        dpxs.append(dpx)
        dpys.append(dpy)
        dts.append(t)

        done = bool(out[2][0] or out[3][0])
        if done:
            reset_env(venv)

    venv.close()

    return {
        "dx": np.array(dxs),
        "dy": np.array(dys),
        "dz": np.array(dzs),
        "dpx": np.array(dpxs),
        "dpy": np.array(dpys),
        "dt": np.array(dts),
    }

def suggest_goal_bounds(data, pct_min=1, pct_max=99, slack=1.2):
    low, high = {}, {}
    for key, values in data.items():
        lo = np.percentile(values, pct_min)
        hi = np.percentile(values, pct_max)
        center = 0.5 * (lo + hi)
        width = (hi - lo) * slack
        low[key]  = center - 0.5 * width
        high[key] = center + 0.5 * width
    return low, high

if __name__ == "__main__":
    cfg = Config()
    print("Collecting relative offsets …")
    data = collect_relative_offsets(cfg, num_steps=30000)

    low, high = suggest_goal_bounds(data, pct_min=1, pct_max=99, slack=1.2)
    print("=== Suggested goal bounds ===")
    print("goal_low  =", [
        low["dx"], low["dy"], low["dz"],
        low["dpx"], low["dpy"], 0.2
    ])
    print("goal_high =", [
        high["dx"], high["dy"], high["dz"],
        high["dpx"], high["dpy"], 0.8
    ])

100%|██████████| 30000/30000 [05:33<00:00, 90.02it/s]

=== Suggested goal bounds ===
goal_low  = [2.6226827461719515, 0.09191729348897937, -0.27048435640335083, 2.733618104457855, -0.573699263960123, 0.2]
goal_high = [3.141523899316788, 1.236468253314495, -0.15279872941970826, 3.25203111410141, 0.5706091401278972, 0.8]


In [1]:
from stable_baselines3.common.vec_env import DummyVecEnv
from hrl.worker_env import TableTennisWorker
from config import Config

venv = DummyVecEnv([lambda: TableTennisWorker(Config())])
obs = venv.reset()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {ne

MyoSuite:> Registering Myo Envs
    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


In [2]:
venv.step(venv.action_space.sample())

IndexError: invalid index to scalar variable.

In [ ]:
def draw_paddle_normal(sim, paddle_pos, paddle_n, length=0.15):
    """
    Draw a line showing the paddle normal in the viewer.
    """
    start = paddle_pos
    end = paddle_pos + length * paddle_n

    sim.add_marker(
        pos=start,
        type=sim.mjtGeom.mjGEOM_ARROW,
        size=[length, 0.01, 0.01],
        rgba=[1, 0, 0, 1],  # red arrow
        mat=None,
        label=""
    )

In [6]:
import numpy as np
from myosuite.utils import gym
from utils import quat_to_paddle_normal

In [ ]:
from myosuite.utils import gym

env = gym.make("myoChallengeTableTennisP2-v0")
obs, info = env.reset()

for i in range(5):
    env.step(env.action_space.sample())

    origin = env.sim.data.site_xpos[
        env.sim.model.site_name2id("paddle_origin")
    ]
    tip = env.sim.data.site_xpos[
        env.sim.model.site_name2id("paddle_normal_tip")
    ]

    n = tip - origin
    n /= np.linalg.norm(n)
    print(f"[step {i}] site normal:", n)